# Constrained Declarative Node Worked Examples

In this notebook we explore three simple examples of constrained declarative nodes.
The first example solves a problem with linear objective and unit sphere constraint,
the second example solves a problem with quadratic objective and unit sphere constraint,
and the third example solve a problem with quadratic objective and unit ball constraint.
All examples have two-dimensional output to allow for easy visualization.

It is assumed that you have read the ["Deep Declarative Networks: A New Hope"](https://arxiv.org/abs/1909.04866) paper.

In [1]:
%matplotlib inline

In [2]:
import autograd.numpy as np
from autograd import grad
from autograd import jacobian

import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML

import warnings
warnings.filterwarnings('ignore')

## Example 1: Minimize linear objective over unit circle

Consider the problem
$$
\begin{array}{rll}
y(x) =& \text{argmin}_u & (1, x)^T u
\\
& \text{subject to} & \|u\|_2^2 = 1
\end{array}
$$
where $x \in \mathbb{R}$ and $y(x) \in \mathbb{R}^2$.

This problem has an analytic solution
$$
\begin{align*}
y &= \frac{-1}{\sqrt{1 + x^2}} \begin{bmatrix} 1 \\ x \end{bmatrix} \\
\text{D}y &= \begin{bmatrix} \frac{\text{d}y_1}{\text{d}x} \\ \frac{\text{d}y_2}{\text{d}x} \end{bmatrix}
= \frac{1}{(1 + x^2)^{3/2}} \begin{bmatrix} x \\ -1 \end{bmatrix}
\end{align*}
$$
which we have implemented in the code below.

In [3]:
# objective and constraint functions

def f(x, u):
    """Objective function taking x \in \reals, u \in \reals^2."""
    return np.dot(u, np.array([1.0, x]))

def h(u):
    """Constraint function taking u \in \reals^2."""
    return np.dot(u, u) - 1.0

# analytical solutions

def solve(x):
    """Analytical solution to min. f s.t. h = 0. Returns both optimal primal and dual variables."""
    return -1.0 / np.sqrt(1.0 + x**2.0) * np.array([1.0, x]), -0.5 * np.sqrt(1 + x**2.0)

def dy_star(x):
    """Analytical derivative of y with respect to x."""
    return 1.0 / np.power(1 + x**2.0, 1.5) * np.array([x, -1.0])

We can also compute the gradient using implicit differentiation on the first-order optimality condition of the Lagrangian,

$$
\begin{align*}
0 &= 
\text{D} \begin{bmatrix}
    \text{D}_{Y} {\cal L}(x, y, \lambda) \\
    \text{D}_{\Lambda} {\cal L}(x, y, \lambda)
\end{bmatrix}
\\
&=
\text{D} \begin{bmatrix}
    \text{D}_Y f(x, y) - \lambda \text{D}_Y h(y) \\
    h(y)
\end{bmatrix}
\\
&=
\begin{bmatrix}
    \text{D}^2_{XY} f(x, y) + \text{D}^2_{YY} f(x, y) \text{D}y - \lambda \text{D}^2_{YY} h(y) \text{D}y - (\text{D}_Y h(y))^T \text{D}\lambda \\
    \text{D}_{Y} h(y) \text{D}y
\end{bmatrix}
\\
&=
\begin{bmatrix}
    \text{D}^2_{XY} f(x, y) \\
    0
\end{bmatrix}
+
\begin{bmatrix}
    \text{D}^2_{YY} f(x, y) - \lambda \text{D}^2_{YY} h(y) & -(\text{D}_Y h(y))^T \\
    \text{D}_{Y} h(y) & 0
\end{bmatrix}
\begin{bmatrix}
    \text{D}y \\
    \text{D}\lambda
\end{bmatrix}
\\
&=
\begin{bmatrix}
    b \\
    0
\end{bmatrix}
+
\begin{bmatrix}
    H & -a \\
    a^T & 0
\end{bmatrix}
\begin{bmatrix}
    \text{D}y \\
    \text{D}\lambda
\end{bmatrix}
\end{align*}
$$

Block elimination then gives

$$
\begin{align*}
\text{D} y
&= \left( \frac{H^{-1}aa^TH^{-1}}{a^T H^{-1} a} - H^{-1}\right) b \\
&= \frac{v^T b}{v^T a} v - w
\end{align*}
$$

where $v = H^{-1}a$ and $w = H^{-1}b$, and which we have implemented below using automatic differentiation.

In [4]:
# gradient by implicit differentiation

fY = grad(f, 1)
hY = grad(h)
fXY = jacobian(fY, 0)
fYY = jacobian(fY, 1)
hYY = jacobian(hY, 0)

def dy(x):
    """Compute gradient of y with respect to x using implicit differentiation."""

    y, nu = solve(x)

    # Here we solve a system of linear equations rather than inverting H. The linear
    # algebra solver gives $w = H^{-1} D_Y h$ and $v = H^{-1} D^2_{XY} f$ from which
    # we compute Dy(x) as (w^T D^2_{XY} f / w^T D_{Y} h) w - v. 

    a = hY(y)
    b = fXY(x, y)
    H = fYY(x, y) - nu * hYY(y)
    
    (v, w) = np.linalg.solve(H, np.stack((a, b)))
    return v.dot(b) / v.dot(a) * v - w

The following compares the analytic gradient against the gradient derived implicitly over the range $x \in [-2, 2]$.

In [5]:
# generate data for different input
x = np.linspace(-2.0, 2.0, num=51)
y = [solve(xi)[0] for xi in x]
dy_analytic = [dy_star(xi) for xi in x]
dy_implicit = [dy(xi) for xi in x]

# print difference between analytic and implicit gradients
err = np.abs(np.array(dy_analytic) - np.array(dy_implicit))
print("Max. difference between implicit and analytic gradients is {}".format(np.max(err)))

Max. difference between implicit and analytic gradients is 1.1102230246251565e-16


### Animation

To get a better understanding of how the output of this constrained declarative node changes with input---a requirement for end-to-end learning---we animate the geometry of the solution to the problem $y$ as we vary the input $x$. Along with the animation we plot both $y$ and $\text{D}y$ as functions of $x$.

In [6]:
# simple animation of the problem

def init():
    for a in ax:
        a.axis('square')
        a.set_xlim(x[0], x[-1])    
    ax[0].set_ylim(x[0], x[-1])
    ax[1].set_ylim(-1.0, 1.0)
    ax[2].set_ylim(-1.0, 1.0)

    return ax


def animate(fnum, x, y, dy):
    
    for a in ax:
        a.clear()
        
    ax[0].annotate("", xy=(1.0, x[fnum]), xytext=(0, 0), arrowprops = dict(arrowstyle="->", linewidth=3, color='b'))
    for delta in np.linspace(-2.0, 2.0, 5):
        dx = delta * np.sqrt(1.0 + x[fnum]**2.0)
        ax[0].plot([-2.0 * x[fnum] + dx, 2.0 * x[fnum] + dx], [2.0, -2.0], 'b--', linewidth=1)

    ax[0].plot(np.cos(np.linspace(0.0, 2.0 * np.pi)), np.sin(np.linspace(0.0, 2.0 * np.pi)), 'r--', linewidth=1)
    ax[0].plot(y[fnum][0], y[fnum][1], 'ro', markersize=12, linewidth=2)
    ax[0].set_xlabel(r"$u_1$"); ax[0].set_ylabel(r"$u_2$")

    ax[1].plot(x[0:fnum], [yi[0] for yi in y[0:fnum]], x[0:fnum], [yi[1] for yi in y[0:fnum]])
    ax[1].legend([r"$y_1$", r"$y_2$"])
    
    ax[2].plot(x[0:fnum], [di[0] for di in dy[0:fnum]], x[0:fnum], [di[1] for di in dy[0:fnum]])
    ax[2].legend([r"$Dy_1$", r"$Dy_2$"], loc='upper left')
    
    for a in ax:
        a.axis('square')
        a.set_xlim(x[0], x[-1])
    ax[0].set_ylim(x[0], x[-1])
    ax[1].set_ylim(-1.0, 1.0)
    ax[2].set_ylim(-1.0, 1.0)
    
    return ax


fig = plt.figure()
ax = [plt.subplot(1, 2, 1), plt.subplot(2, 2, 2), plt.subplot(2, 2, 4)]
plt.suptitle(r"$y$ = argmin $(1, x)^T u$ subject to $\|u\|^2 = 1$")

ani = animation.FuncAnimation(fig, animate, init_func=init, fargs=(x, y, dy_implicit),
                              interval=100, frames=len(x), blit=False, repeat=False)

plt.close(fig)

In [ ]:
# display using video or javascript

HTML(ani.to_html5_video())
#HTML(ani.to_jshtml())

## Example 2: Minimize quadratic objective over unit circle

Consider the problem
$$
\begin{array}{rll}
y(x) =& \text{argmin}_u & \frac{1}{2} u^T u - x^T u
\\
& \text{subject to} & \|u\|_2^2 = 1
\end{array}
$$
with $x \in \mathbb{R}^2$ and $y \in \mathbb{R}^2$.

This is just the Euclidean projection of the two-dimensional point $x$ onto the unit cirle, which can be seen by replacing the objective function with $\frac{1}{2} \|u - x\|^2 = \frac{1}{2} u^T u - x^T u + \frac{1}{2} x^T x$ and recognizing that $\frac{1}{2} x^T x$ plays no part in the optimization. The problem has analytic solution
$$
\begin{align*}
y &= \frac{1}{\|x\|} x \\
\text{D} y &= \begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} \\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2}
\end{bmatrix}
= \frac{1}{\|x\|^3} \begin{bmatrix}
    x_2^2 & - x_1 x_2 \\
    - x_1 x_2 & x_1^2
\end{bmatrix}
\end{align*}
$$


In [8]:
# objective and constraint functions

def f(x, u):
    """Objective function taking x \in \reals^2, u \in \reals^2."""
    return 0.5 * np.dot(u, u) - np.dot(u, x)

def h(u):
    """Constraint function taking u \in \reals^2."""
    return np.dot(u, u) - 1.0

# analytical solutions

def solve(x):
    """Analytical solution to min. f s.t. h = 0. Returns both optimal primal and dual variables."""
    return 1.0 / np.sqrt(np.dot(x, x)) * x, None

def dy_star(x):
    """Analytical derivative of y with respect to x."""
    return 1.0 / np.power(np.dot(x, x), 1.5) * np.array([[x[1] ** 2, -x[0]*x[1]], [-x[0]*x[1], x[0] ** 2]])

Again, we can also compute the gradient using implicit differentiation. Now we have two-dimensional inputs and outputs. We also assume that our solver does not return $\lambda$ and so have to calculate it in the gradient function as

$$
\lambda = \frac{\text{D}_Y f(x, y)_i}{\text{D}_Y h(y)_i}
$$
for any $i$ such that $\text{D}_Y h(y)_i \neq 0.$

The gradient is
$$
\begin{align*}
\text{D} y
&= \left( \frac{H^{-1}aa^TH^{-1}}{a^T H^{-1} a} - H^{-1}\right) B \\
&= \frac{1}{v^T a} v v^T B - W
\end{align*}
$$
where $v = H^{-1}a$ and $W = H^{-1}B$, and which we have implemented below using automatic differentiation.

In [9]:
# gradient by implicit differentiation

fY = grad(f, 1)
hY = grad(h)
fXY = jacobian(fY, 0)
fYY = jacobian(fY, 1)
hYY = jacobian(hY, 0)

def dy(x):
    """Compute gradient of y with respect to x using implicit differentiation."""

    y, nu = solve(x)
    
    a = hY(y)
    b = fXY(x, y)
    if (nu is None):
        indx = np.nonzero(a)[0]
        nu = 0 if (len(indx) == 0) else fY(x, y)[indx[0]] / a[indx[0]] 

    H = fYY(x, y) - nu * hYY(y)
    
    # Here we solve a system of linear equations rather than inverting H.
    w = np.linalg.solve(H, np.concatenate((a.reshape((len(y), 1)), b), axis=1))
    return (np.outer(w[:, 0], w[:, 0].dot(b) / w[:, 0].dot(a)) - w[:, 1:len(x) + 1]).squeeze()

In [10]:
# generate data for different input
x = np.linspace(-2.0, 2.0, num=51)
y = [solve(np.array([0.5, xi]))[0] for xi in x]
dy_analytic = [dy_star(np.array([0.5, xi])) for xi in x]
dy_implicit = [dy(np.array([0.5, xi])) for xi in x]

# print difference between analytic and implicit gradients
err = np.abs(np.array(dy_analytic) - np.array(dy_implicit))
print("Max. difference between implicit and analytic gradients is {}".format(np.max(err)))

Max. difference between implicit and analytic gradients is 4.996003610813204e-16


In [11]:
# simple animation of the problem

def init():
    for a in ax:
        a.axis('square')
        a.set_xlim(x[0], x[-1])    
    ax[0].set_ylim(x[0], x[-1])
    ax[1].set_ylim(-1.0, 1.0)
    ax[2].set_ylim(-1.0, 1.0)

    return ax


def animate(fnum, x, y, dy):
    
    for a in ax:
        a.clear()

    ax[0].annotate("", xy=(-0.5, -x[fnum]), xytext=(0, 0), arrowprops=dict(arrowstyle="->", linewidth=3, color='b'))
    for r in [0.125, 0.5, 1.125, 2.0, 3.125]:
        ax[0].plot(r * np.cos(np.linspace(0.0, 2.0 * np.pi)) + 0.5, r * np.sin(np.linspace(0.0, 2.0 * np.pi)) + x[fnum], 'b--', linewidth=1)

    ax[0].plot(np.cos(np.linspace(0.0, 2.0 * np.pi)), np.sin(np.linspace(0.0, 2.0 * np.pi)), 'r--', linewidth=1)
    ax[0].plot(y[fnum][0], y[fnum][1], 'ro', markersize=12, linewidth=2)        
    ax[0].set_xlabel(r"$u_1$"); ax[0].set_ylabel(r"$u_2$")

    ax[1].plot(x[0:fnum], [yi[0] for yi in y[0:fnum]], x[0:fnum], [yi[1] for yi in y[0:fnum]])
    ax[1].legend([r"$y_1$", r"$y_2$"])
    
    ax[2].plot(x[0:fnum], [di[0, 0] for di in dy[0:fnum]], x[0:fnum], [di[0, 1] for di in dy[0:fnum]],
               x[0:fnum], [di[1, 0] for di in dy[0:fnum]], x[0:fnum], [di[1, 1] for di in dy[0:fnum]])
    ax[2].legend([r"$Dy_{1,1}$", r"$Dy_{1,2}$", r"$Dy_{2,1}$", r"$Dy_{2,2}$"], loc='upper left')
    ax[2].set_xlabel(r"$x_2$");
    
    for a in ax:
        a.axis('square')
        a.set_xlim(x[0], x[-1])
    ax[0].set_ylim(x[0], x[-1])
    ax[1].set_ylim(-1.0, 1.0)
    ax[2].set_ylim(-1.0, 1.0)
    
    return ax


In [ ]:
fig = plt.figure()
ax = [plt.subplot(1, 2, 1), plt.subplot(2, 2, 2), plt.subplot(2, 2, 4)]
plt.suptitle(r"$y$ = argmin $\frac{1}{2} u^T u - x^T u$ subject to $\|u\|^2 = 1$")

ani = animation.FuncAnimation(fig, animate, init_func=init, fargs=(x, y, dy_analytic),
                              interval=100, frames=len(x), blit=False, repeat=False)

plt.close(fig)

# display using video or javascript
HTML(ani.to_html5_video())
#HTML(ani.to_jshtml())

## Example 3: Minimize quadratic objective over unit ball

Consider the problem
$$
\begin{array}{rll}
y(x) =& \text{argmin}_u & \frac{1}{2} u^T u - x^T u
\\
& \text{subject to} & \|u\|_2^2 \leq 1
\end{array}
$$

The problem has analytic solution
$$
\begin{align*}
y &= \begin{cases}
x & \text{if $\|x\| \leq 1$} \\
\frac{1}{\|x\|} x & \text{otherwise}
\end{cases}
\\
\text{D} y &= \begin{cases}
I & \text{if $\|x\| < 1$} \\
\frac{1}{\|x\|^3} \begin{bmatrix}
    x_2^2 & - x_1 x_2
    \\
    - x_1 x_2 & x_1^2
\end{bmatrix}
& \text{if $\|x\| > 1$}
\end{cases}
\end{align*}
$$

In [13]:
# objective and constraint functions

def f(x, u):
    """Objective function taking x \in \reals^2, u \in \reals^2."""
    return 0.5 * np.dot(u, u) - np.dot(u, x)

def h(u):
    """Constraint function taking u \in \reals^2."""
    return np.dot(u, u) - 1.0

# analytical solutions

def solve(x):
    """Analytical solution to min. f s.t. h = 0. Returns both optimal primal and dual variables."""
    d = np.dot(x, x)
    return x.copy() if (d <= 1.0) else 1.0 / np.sqrt(d) * x, None

def dy_star(x):
    """Analytical derivative of y with respect to x."""
    d = np.dot(x, x)
    return np.eye(2, 2) if (d < 1.0) else 1.0 / np.power(d, 1.5) * np.array([[x[1] ** 2, -x[0]*x[1]], [-x[0]*x[1], x[0] ** 2]])

In [14]:
# gradient by implicit differentiation

fY = grad(f, 1)
hY = grad(h)
fXY = jacobian(fY, 0)
fYY = jacobian(fY, 1)
hYY = jacobian(hY, 0)

def dy(x):
    """Compute gradient of y with respect to x using implicit differentiation."""

    y, nu = solve(x)
    
    # use unconstrained gradient
    if (h(x) <= 0.0):
        return -1.0 * np.linalg.solve(fYY(x, y), fXY(x, y))
    
    # use constrained gradient
    a = hY(y)
    b = fXY(x, y)
    if (nu is None):
        indx = np.nonzero(a)[0]
        nu = 0.0 if (len(indx) == 0) else fY(x, y)[indx[0]] / a[indx[0]] 

    H = fYY(x, y) - nu * hYY(y)
    
    # Here we solve a system of linear equations rather than inverting H.
    w = np.linalg.solve(H, np.concatenate((a.reshape((len(y), 1)), b), axis=1))
    return (np.outer(w[:, 0], w[:, 0].dot(b) / w[:, 0].dot(a)) - w[:, 1:len(x) + 1]).squeeze()

In [15]:
# generate data for different input
x = np.linspace(-2.0, 2.0, num=51)
y = [solve(np.array([0.5, xi]))[0] for xi in x]
dy_analytic = [dy_star(np.array([0.5, xi])) for xi in x]
dy_implicit = [dy(np.array([0.5, xi])) for xi in x]

# print difference between analytic and implicit gradients
err = np.abs(np.array(dy_analytic) - np.array(dy_implicit))
print("Max. difference between implicit and analytic gradients is {}".format(np.max(err)))

Max. difference between implicit and analytic gradients is 1.942890293094024e-16


In [ ]:
fig = plt.figure()
ax = [plt.subplot(1, 2, 1), plt.subplot(2, 2, 2), plt.subplot(2, 2, 4)]
plt.suptitle(r"$y$ = argmin $\frac{1}{2} u^T u - x^T u$ subject to $\|u\|^2 \leq 1$")

ani = animation.FuncAnimation(fig, animate, init_func=init, fargs=(x, y, dy_analytic),
                              interval=100, frames=len(x), blit=False, repeat=False)

plt.close(fig)

# display using video or javascript
HTML(ani.to_html5_video())
#HTML(ani.to_jshtml())